# Natural Gas Pricing Model


### Problem Statement:

- Write a function to return total value of a gas storage contract.

#### Components to consider:

1. Maximum storage capacity.
2. Storage costs per day.
3. Injection/Withdrawal cost per unit of gas.
4. Injection Dates and Prices.
5. Withdrawal Dates and Prices.

#### Limitations to consider:

1. Injection should not exceed Total Maximum storage capacity.
2. Withdrawal should not exceed Total Stored Gas at that point or Maximum storage capacity.

#### Pseudo Code

1. We need to keep track of number of days the gas units are stored at each step.(Storage costs need to be updated at each Injection/Withdrawal date)
2. Rough forumula :   Contract Value= Price_Withdrawal*Withdrawal_Volume-Storage_costs- Injection Costs(to date)-Withdrawal Costs(to date).   
3. While calculating gas stored at each step and keeping it less than or equal to Maximum storage capacity.

In [1]:
from datetime import datetime


def storage_contract_valuation(withdrawal_dates, injection_dates, prices_injection, prices_withdrawal,
                    withdrawal_costs_per_unit, injection_costs_per_unit,
                    maximum_storage_capacity, injection_volume, 
                    withdrawal_volume, storage_costs_per_day):
# Goal: To calculate value of gas storage contract.

# Parameters
# withdrawal_dates: List of withdrawal dates. 
# injection_dates: List of injection dates. 
# prices_injection: Gas prices on corresponding injection date.
# prices_withdrawal: Gas prices on corresponding withdrawal date.
# withdrawal_costs_per_unit: Cost of withdrawing one unit of gas. 
# injection_costs_per_unit: Cost of injecting one unit of gas.
# maximum_storage_capacity: Maximum volume of gas that can be stored.
# injection_volume: Units injected on corresponding injection date.
# withdrawal_volume: Units withdrawn on corresponding withdrawal date.
# storage_costs_per_day: Cost of storing 1 unit of gas per day.

# Returns
# Total value of the contract


    # Converting injection dates and withdrawal dates to datetime format for easier calculation of storage days
    injection_dates = [datetime.strptime(date, "%Y-%m-%d") for date in injection_dates]
    withdrawal_dates = [datetime.strptime(date, "%Y-%m-%d") for date in withdrawal_dates]

    current_volume = 0  # current volume of gas available for withdrawal/stored.
    total_revenue = 0
    total_costs = 0
    storage = []  # Tracking injection volume and its injection date.
    
    #sorting in order to track the storage duration in order of events.
    all_dates = sorted(set(injection_dates+withdrawal_dates))
    
    # looping through all dates and processing injections and withdrawals
    for date in all_dates:
        # Handling Injections
        if date in injection_dates:
            idx = injection_dates.index(date)
            volume_to_inject = injection_volume[idx]
            price_at_injection = prices_injection[idx]
            
            # Making sure to not exceed maximum storage capacity while injecting
            if current_volume + volume_to_inject > maximum_storage_capacity:
                volume_to_inject = maximum_storage_capacity- current_volume  # inject only avaiable volume
            
            current_volume += volume_to_inject
            total_costs += volume_to_inject * injection_costs_per_unit # Cost of injecting gas
            total_costs += volume_to_inject * price_at_injection  # Cost of buying gas
            
            # Adding a record for gas injected to storage
            
            storage.append((volume_to_inject, date))
#             print(storage)
        # Handling Withdrawals
        if date in withdrawal_dates:
            
            idx = withdrawal_dates.index(date)
            volume_to_withdraw = withdrawal_volume[idx]
            price_at_withdrawal = prices_withdrawal[idx]
            
            # Making sure we have enough volume to withdraw
            if current_volume < volume_to_withdraw:
                volume_to_withdraw = current_volume  # withdraw all available gas
                
            current_volume -= volume_to_withdraw
            total_costs += volume_to_withdraw * withdrawal_costs_per_unit  # Cost of withdrawing gas
            total_revenue += volume_to_withdraw * price_at_withdrawal  # Revenue for selling gas
            
            # Calculating storage cost for withdrawn gas, utilizing oldest injections first
            
            gas_left_to_withdraw = volume_to_withdraw
            for i in range(len(storage)):
                injected_volume, injected_date=storage[i]
                if gas_left_to_withdraw == 0:
                    break
                if injected_volume <= gas_left_to_withdraw:
                    # Calculate storage cost for this volume
                    storage_duration = (date - injected_date).days
                    total_costs += injected_volume * storage_costs_per_day * storage_duration
                    gas_left_to_withdraw -= injected_volume
                    storage[i]=(0,injected_date)    # All withdrawn from this injection
                else:
                    #Partially withdrawn from this injection
                    storage_duration = (date - injected_date).days
                    total_costs += gas_left_to_withdraw * storage_costs_per_day * storage_duration
                    storage[i]=(injected_volume - gas_left_to_withdraw,injected_date)
                    gas_left_to_withdraw = 0  # setting gas left to withdraw to 0 as no more gas left in storage
#                 print(storage)
            #Removing injection records that have been withdrawn completely
            storage=[(v,d) for v,d in storage if v>0]

    # Final contract valuation
    
    total_value= total_revenue-total_costs
    
    return total_value
    
        
    
    
    
    
    

In [2]:
### Testing the function with sample inputs

# Sample Inputs
injection_dates = ["2024-01-01"]
withdrawal_dates = ["2024-02-01"]
prices_injection = [50]  # Gas prices when buying (injecting)
prices_withdrawal = [80]  # Gas prices when selling (withdrawing)
injection_costs_per_unit = 2  # Cost per unit of gas injected
withdrawal_costs_per_unit = 1  # Cost per unit of gas withdrawn
injection_volume = [100]  # Amounts of gas injected
withdrawal_volume = [100]  # Amounts of gas withdrawn
maximum_storage_capacity = 150  # Maximum storage capacity
storage_costs_per_day = 0.5  # Storage cost per day per unit of gas

# Price the contract
contract_value = 0 
contract_value = storage_contract_valuation(
                    withdrawal_dates, injection_dates, 
                    prices_injection, prices_withdrawal,
                    withdrawal_costs_per_unit, injection_costs_per_unit,
                    maximum_storage_capacity, injection_volume, 
                    withdrawal_volume, storage_costs_per_day)

manual_contract_value= (80*100)-(50 * 100 + 100 * 2 + 1 * 100 + 31 * 100 * 0.5)

print(f"Total Contract Value: {contract_value}")
print(f"Manually Calculated Contract Value: {manual_contract_value}")

[(100, datetime.datetime(2024, 1, 1, 0, 0))]
Total Contract Value: 1150.0
Manually Calculated Contract Value: 1150.0


In [3]:
### Testing the function with sample inputs

# Sample Inputs
injection_dates = ["2024-01-01", "2024-01-10"]
withdrawal_dates = ["2024-02-01", "2024-02-15"]
prices_injection = [50, 55]  # Gas prices when buying (injecting)
prices_withdrawal = [70, 75]  # Gas prices when selling (withdrawing)
injection_costs_per_unit = 2  # Cost per unit of gas injected
withdrawal_costs_per_unit = 1  # Cost per unit of gas withdrawn
injection_volume = [100, 50]  # Amounts of gas injected
withdrawal_volume = [80, 70]  # Amounts of gas withdrawn
maximum_storage_capacity = 150  # Maximum storage capacity
storage_costs_per_day = 0.5  # Storage cost per day per unit of gas

# Price the contract
contract_value=0
contract_value = storage_contract_valuation(
                    withdrawal_dates, injection_dates, 
                    prices_injection, prices_withdrawal,
                    withdrawal_costs_per_unit, injection_costs_per_unit,
                    maximum_storage_capacity, injection_volume, 
                    withdrawal_volume, storage_costs_per_day)

manual_contract_value= (70 * 80 + 75 * 70)-(50 * 100+ 55 * 50+ 100 * 2 + 50 * 2+
                                           80 * 1 + 70 * 1 + 80 * 31 * 0.5 + 20 * 45 * 0.5 + 50 *36*0.5)

print(f"Total Contract Value: {contract_value}")
print(f"Manually Calculated Contract Value: {manual_contract_value}")

[(100, datetime.datetime(2024, 1, 1, 0, 0))]
[(100, datetime.datetime(2024, 1, 1, 0, 0)), (50, datetime.datetime(2024, 1, 10, 0, 0))]
Total Contract Value: 60.0
Manually Calculated Contract Value: 60.0
